# New York Best Selling Books

This notebook documents the data wrangling process for the New York Times best-selling books dataset obtained from the New York Times API(BooksAPI). Additionally, it includes integration with the Google Books API for supplementary information when needed.

### Table Of Contents:

- [Basic Info](#Basic-info-about-the-data-present-in-the-API:)

- [Categories of best sellers list](#Categories-of-best-sellers-list)

- [API Calls](#API-Calls)

    1.[Best Sellers List](#Best-Sellers-List)
    
    2.[Best Sellers List by Date](#Best-Sellers-List-by-Date)
    
    3.[Best Sellers History](#Best-Sellers-History)
    
    4.[Best Seller Full Overview](#Best-Seller-Full-Overview)
    
    5.[Best Sellers List Names](#Best-Sellers-List-Names)
    
    6.[Top 5 Best Sellers list for Specified date (Overview)](#Top-5-Best-Sellers-list-for-Specified-date-(Overview))



In [2]:
#"api-key": "VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"

Importing the required packages

In [3]:
import requests, json
import pandas as pd
import numpy as np

## Basic info about the data present in the API:


#### 1.  List Data
The service returns the books on the best sellers list for the specified date and list name.

- *lists/{date}/{name}*

For current list:

- */lists/current/hardcover-fiction.json* 

#### 2.  Book reviews
The book reviews service lets you get NYT book review by author, ISBN, or title.

- */reviews.json?author=Michelle+Obama*

- */reviews.json?isbn=9781524763138*

- */reviews.json?title=Becoming*


#### 3.  Overview Response
Get all books for all the Best Sellers lists for specified date.

- */lists/full-overview.json*

Get top 5 books for all the Best Sellers lists for specified date.

- */lists/overview.json*

#### 4.  Example Calls
- *https://api.nytimes.com/svc/books/v3/lists/current/hardcover-fiction.json?api-key=yourkey*

- *https://api.nytimes.com/svc/books/v3/reviews.json?author=Stephen+King&api-key=yourkey*


### Categories of best sellers list

1. combined-print-and-e-book-fiction - **combined-print-and-e-book-fiction**
2. combined-print-and-e-book-nonfiction - **combined-print-and-e-book-nonfiction**
3. hardcover-fiction - **hardcover-fiction**
4. hardcover-nonfiction - **hardcover-nonfiction**
5. trade-fiction-paperback - **trade-fiction-paperback**
6. paperback-nonfiction - **paperback-nonfiction**
7. advice-how-to-and-miscellaneous - **advice-how-to-and-miscellaneous**
8. Childrens Middle Grade Hardcover - **childrens-middle-grade-hardcover**
9. Picture Books - **picture-books**
10. Series Books - **series-books**
11. Young Adult Hardcover - **young-adult-hardcover**
12. Audio Fiction - **audio-fiction	**
13. Audio Nonfiction - **audio-nonfiction**
14. Business Books - **business-books**
15. graphic-books-and-manga - **graphic-books-and-manga**
16. mass-market-monthly - **mass-market-monthly**
17. middle-grade-paperback-monthly - **middle-grade-paperback-monthly**
18. Young Adult Paperback Monthly - **young-adult-paperback-monthly**

## API Calls

Let's start by figuring out each GET method:

### Best Sellers List
**---------------------------------**

Get Best Sellers list. If no date is provided returns the latest list.

> ***/lists.json***

Parameters:

- **list (required,String)**: The name of the Times best sellers list (default:hardcover-fiction).

*Note this paramater is not working in the actual API.(Checked on the API page as well. Hence excluding it)*
*- **bestsellers-date (optional, YYYY-MM-DD):** The week-ending date for the sales reflected on list-name.The bestsellers-date may be significantly earlier than published-date.*

- **published-date (optional, YYYY-MM-DD):** The date the best sellers list was published on NYTimes.com (different than bestsellers-date).Important note here. The published date you give will produce the results for future date if the list has not been published on the user input date

- **offset (optional, integer):** Must be a multiple of 20. Sets the starting point of the result set (0, 20, ...). Used to paginate thru books if list has more than 20. Defaults to 0. The num_results field indicates how many books are in the list.

In [17]:
#Function to print result from API call
def get_best_list(list_name, published_date=None, offset=None):

        
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/lists.json"
    
    #API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"
    
    # Request parameters
    params = {
        "list": list_name,
        "api-key": api_key
    }
    
    if published_date:
        params["published-date"] = published_date
    if offset is not None:
        params["offset"] = offset
    
    # Making the API request
    response = requests.get(base_url, params=params)
    
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [19]:
#Function call
best_list = get_best_list(list_name="hardcover-fiction",published_date="2024-04-28")
best_list

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 15,
 'last_modified': '2024-04-17T22:25:30-04:00',
 'results': [{'list_name': 'Hardcover Fiction',
   'display_name': 'Hardcover Fiction',
   'bestsellers_date': '2024-04-13',
   'published_date': '2024-04-28',
   'rank': 1,
   'rank_last_week': 1,
   'weeks_on_list': 10,
   'asterisk': 0,
   'dagger': 0,
   'amazon_product_url': 'https://www.amazon.com/dp/1250178630?tag=thenewyorktim-20',
   'isbns': [{'isbn10': '1250178630', 'isbn13': '9781250178633'},
    {'isbn10': '1250178657', 'isbn13': '9781250178657'}],
   'book_details': [{'title': 'THE WOMEN',
     'description': 'In 1965, a nursing student follows her brother to serve during the Vietnam War and returns to a divided America.',
     'contributor': 'by Kristin Hannah',
     'author': 'Kristin Hannah',
     'contributor_note': '',
     'price': '0.00',
     'age_group': '',
     'publisher': "St. Martin's",
    

Here the dataframe is being created beginning from the key **'results'**

Structure of the json:

- So the entire bestsellers list is inside a list - '[]' which is the value of the key 'results'.
- Each row is inside a dictionary - '{}' which has several key:value pairs.
- Now the keys 'isbns', 'book_details', 'reviews' all have a list - '[]' within which there is a dictionary - '{}' with several key:value pair details. So we need to unravel those as well.
- We will also be dropping the 'list_name', 'display_name'.




In [24]:
# Extracting the 'results' data
results_data = best_list.get('results', [])
# Converting to DataFrame
df = pd.DataFrame(results_data)
# Dropping 'list_name', 'display_name'
df = df.drop(['list_name', 'display_name'], axis=1)
# Extracting book details into separate columns
df = pd.concat([df.drop(['book_details'], axis=1), pd.json_normalize(df['book_details'].apply(lambda x: x[0]))], axis=1)
# Extracting isbns into separate columns
df = pd.concat([df.drop(['isbns'], axis=1), pd.json_normalize(df['isbns'].apply(lambda x: x[0]))], axis=1)
# Extracting reviews into separate columns
df = pd.concat([df.drop(['reviews'], axis=1), pd.json_normalize(df['reviews'].apply(lambda x: x[0]))], axis=1)
# Setting the display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head(2)

,bestsellers_date,published_date,rank,rank_last_week,weeks_on_list,asterisk,dagger,amazon_product_url,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,isbn10,isbn13,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link
0,2024-04-13,2024-04-28,1,1,10,0,0,https://www.amazon.com/dp/1250178630?tag=thene...,THE WOMEN,"In 1965, a nursing student follows her brother...",by Kristin Hannah,Kristin Hannah,,0.00,,St. Martin's,9781250178633,1250178630,1250178630,9781250178633,,,,
1,2024-04-13,2024-04-28,2,0,1,0,0,https://www.amazon.com/dp/125088425X?tag=thene...,THE FAMILIAR,Luzia Cotado encounters dangers when her magic...,by Leigh Bardugo,Leigh Bardugo,,0.00,,Flatiron,9781250884251,125088425X,125088425X,9781250884251,,,,


Putting it together into a single function.

In [22]:
#Function to print the best sellers list
def print_best_list(list_name, published_date=None, offset=None):
    # Getting best list data
    best_list = get_best_list(list_name, published_date, offset)
    if best_list:
        # Extracting the 'results' data
        results_data = best_list.get('results', [])
        # Converting to DataFrame
        df = pd.DataFrame(results_data)
        # Dropping 'list_name', 'display_name'
        df = df.drop(['list_name', 'display_name'], axis=1)
        # Extracting book details into separate columns
        df = pd.concat([df.drop(['book_details'], axis=1), pd.json_normalize(df['book_details'].apply(lambda x: x[0]))], axis=1)
        # Extracting isbns into separate columns
        df = pd.concat([df.drop(['isbns'], axis=1), pd.json_normalize(df['isbns'].apply(lambda x: x[0]))], axis=1)
        # Extracting reviews into separate columns
        df = pd.concat([df.drop(['reviews'], axis=1), pd.json_normalize(df['reviews'].apply(lambda x: x[0]))], axis=1)
        # Setting the display options to show all columns and rows
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
        # Printing the DataFrame
        return df
    else:
        print("Failed to fetch the best list data.")
        return None

In [23]:
#Function call
print_best_list(list_name="hardcover-fiction", published_date="2024-04-28")

,bestsellers_date,published_date,rank,rank_last_week,weeks_on_list,asterisk,dagger,amazon_product_url,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,isbn10,isbn13,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link
0,2024-04-13,2024-04-28,1,1,10,0,0,https://www.amazon.com/dp/1250178630?tag=thene...,THE WOMEN,"In 1965, a nursing student follows her brother...",by Kristin Hannah,Kristin Hannah,,0.00,,St. Martin's,9781250178633,1250178630,1250178630,9781250178633,,,,
1,2024-04-13,2024-04-28,2,0,1,0,0,https://www.amazon.com/dp/125088425X?tag=thene...,THE FAMILIAR,Luzia Cotado encounters dangers when her magic...,by Leigh Bardugo,Leigh Bardugo,,0.00,,Flatiron,9781250884251,125088425X,125088425X,9781250884251,,,,
2,2024-04-13,2024-04-28,3,3,49,0,0,https://www.amazon.com/dp/1649374046?tag=thene...,FOURTH WING,Violet Sorrengail is urged by the commanding g...,by Rebecca Yarros,Rebecca Yarros,,0.00,,Red Tower,9781649374042,1649374046,1649374046,9781649374042,,,,
3,2024-04-13,2024-04-28,4,0,1,0,0,https://www.amazon.com/dp/0593714490?tag=thene...,TOXIC PREY,The 34th book in the Prey series. Letty and Lu...,by John Sandford,John Sandford,,0.00,,Putnam,9780593714492,0593714490,0593714490,9780593714492,,,,
4,2024-04-13,2024-04-28,5,4,23,0,0,https://www.amazon.com/dp/1649374178?tag=thene...,IRON FLAME,The second book in the Empyrean series. Violet...,by Rebecca Yarros,Rebecca Yarros,,0.00,,Red Tower,9781649374172,1649374178,1649374178,9781649374172,,,,
5,2024-04-13,2024-04-28,6,2,2,0,0,https://www.amazon.com/dp/0593296370?tag=thene...,TABLE FOR TWO,A collection of six short stories based in New...,by Amor Towles,Amor Towles,,0.00,,Viking,9780593296370,0593296370,0593296370,9780593296370,,,,
6,2024-04-13,2024-04-28,7,5,4,0,0,https://www.amazon.com/dp/0385550367?tag=thene...,JAMES,A reimagining of “Adventures of Huckleberry Fi...,by Percival Everett,Percival Everett,,0.00,,Doubleday,9780385550369,0385550367,0385550367,9780385550369,,,,
7,2024-04-13,2024-04-28,8,11,7,0,0,https://www.amazon.com/dp/0593599837?tag=thene...,A FATE INKED IN BLOOD,After the secret of her magic to repel attacks...,by Danielle L. Jensen,Danielle L. Jensen,,0.00,,Del Rey,9780593599839,0593599837,0593599837,9780593599839,,,,
8,2024-04-13,2024-04-28,9,14,11,0,0,https://www.amazon.com/dp/1635574102?tag=thene...,HOUSE OF FLAME AND SHADOW,The third book in the Crescent City series. Br...,by Sarah J. Maas,Sarah J. Maas,,0.00,,Bloomsbury,9781635574104,1635574102,1635574102,9781635574104,,,,
9,2024-04-13,2024-04-28,10,6,4,0,0,https://www.amazon.com/dp/0316499676?tag=thene...,THE #1 LAWYER,A criminal defense attorney in Biloxi becomes ...,by James Patterson and Nancy Allen,James Patterson and Nancy Allen,,0.00,,"Little, Brown",9780316499675,0316499676,0316499676,9780316499675,,,,


---


### Best Sellers List by Date
**-----------------------------------------------**

Get Best Sellers list by date.

> ***/lists/{date}/{list}.json***

Note: It does the same thing as the previous one but lets you add the date and category in the url.

Parameters:

- **list (required,String)**: The name of the Times best sellers list (default:hardcover-fiction).

- **date (required, YYYY-MM-DD/current):** The date the best sellers list was published on NYTimes.com. Use "current" to get latest list.

- **offset (optional, integer):** Must be a multiple of 20. Sets the starting point of the result set (0, 20, ...). Used to paginate thru books if list has more than 20. Defaults to 0. The num_results field indicates how many books are in the list.

In [59]:
#Function to print result from API call
def get_best_list_by_date(list_name, published_date=None, offset=None):
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/lists/{date}/{list}.json"
    # API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"
    # Format the URL
    url = base_url.format(date=published_date, list=list_name)
    # Request parameters
    params = {
        "api-key": api_key
    } 
    if offset is not None:
        params["offset"] = offset
    # Making the API request
    response = requests.get(url, params=params)
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        # Raise an exception for error handling
        response.raise_for_status()

In [60]:
best_list_by_date = get_best_list_by_date(list_name="hardcover-fiction", published_date="2024-01-28")
best_list_by_date

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 15,
 'last_modified': '2024-01-17T23:25:19-05:00',
 'results': {'list_name': 'Hardcover Fiction',
  'list_name_encoded': 'hardcover-fiction',
  'bestsellers_date': '2024-01-13',
  'published_date': '2024-01-28',
  'published_date_description': '',
  'next_published_date': '2024-02-04',
  'previous_published_date': '2024-01-21',
  'display_name': 'Hardcover Fiction',
  'normal_list_ends_at': 15,
  'updated': 'WEEKLY',
  'books': [{'rank': 1,
    'rank_last_week': 1,
    'weeks_on_list': 36,
    'asterisk': 0,
    'dagger': 0,
    'primary_isbn10': '1649374046',
    'primary_isbn13': '9781649374042',
    'publisher': 'Red Tower',
    'description': 'Violet Sorrengail is urged by the commanding general, who also is her mother, to become a candidate for the elite dragon riders.',
    'price': '0.00',
    'title': 'FOURTH WING',
    'author': 'Rebecca Yarros',
    'contribu

In [81]:
# Extracting the 'results' key
results_data = best_list_by_date.get('results', {})
# Convert the dictionary to a DataFrame
#from_dict helps to create a datframe from a dict. Here orient= index sets keys to the index(rows) but when you transpose it become columns.
df = pd.DataFrame.from_dict(results_data, orient='index').T
#Dropping 'list_name', 'list_name_encoded'
df = df.drop(['list_name', 'list_name_encoded','corrections'], axis=1)
# # Extracting book details into separate columns
# df = pd.concat([df.drop(['books'], axis=1), pd.json_normalize(df['books'].apply(lambda x: x[0]))], axis=1)

# Exploding ranks_history list of dictionaries into separate rows
df_exploded = df.explode('books')
df_exploded.reset_index(inplace=True)
df_exploded.drop(columns="index",inplace=True)
#Converting ranks_history back to DataFrame and merge with original DataFrame
df_books = pd.json_normalize(df_exploded['books'])
# Combining the two dataFrames
df = pd.concat([df_exploded.drop(columns=['books']), df_books], axis=1)
# Processing buy_links column
#For each dictionary "link" in list "x"  create a new dictionary where key is the value of "name" and value is the value of "url"
df_buy_links = pd.json_normalize(df['buy_links'].apply(lambda x: {link['name']: link['url'] for link in x}))
# Process isbns column
df_isbns = pd.json_normalize(df['isbns'].apply(lambda x: x[0]))
# Concatenate processed columns with original DataFrame
df = pd.concat([df.drop(['buy_links', 'isbns'], axis=1), df_buy_links, df_isbns], axis=1)
df.head(2)

,bestsellers_date,published_date,published_date_description,next_published_date,previous_published_date,display_name,normal_list_ends_at,updated,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,book_uri,Amazon,Apple Books,Barnes and Noble,Books-A-Million,Bookshop,IndieBound,isbn10,isbn13
0,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,1,1,36,0,0,1649374046,9781649374042,Red Tower,Violet Sorrengail is urged by the commanding g...,0.00,FOURTH WING,Rebecca Yarros,by Rebecca Yarros,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374046?tag=thene...,,,,,,nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a,https://www.amazon.com/dp/1649374046?tag=thene...,https://goto.applebooks.apple/9781649374042?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374042,https://www.indiebound.org/book/9781649374042?...,1649374046,9781649374042
1,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,2,2,10,0,0,1649374178,9781649374172,Red Tower,The second book in the Empyrean series. Violet...,0.00,IRON FLAME,Rebecca Yarros,by Rebecca Yarros,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374178?tag=thene...,,,,,,nyt://book/d3c570c9-3c3a-5c8b-a740-85ea5e92bfc9,https://www.amazon.com/dp/1649374178?tag=thene...,https://goto.applebooks.apple/9781649374172?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374172,https://www.indiebound.org/book/9781649374172?...,1649374178,9781649374172


Converting into a single function

In [85]:
def print_best_list_by_date(list_name, published_date=None, offset=None):
    # Get best list by date data
    best_list_by_date_data = get_best_list_by_date(list_name, published_date, offset)
    if best_list_by_date_data:
        # Extracting the 'results' key
        results_data = best_list_by_date_data.get('results', {})
        # Convert the dictionary to a DataFrame
        df = pd.DataFrame.from_dict(results_data, orient='index').T
        # Dropping 'list_name', 'list_name_encoded'
        df = df.drop(['list_name', 'list_name_encoded', 'corrections'], axis=1)
        # Exploding ranks_history list of dictionaries into separate rows
        df_exploded = df.explode('books')
        df_exploded.reset_index(inplace=True)
        df_exploded.drop(columns="index", inplace=True)
        # Converting ranks_history back to DataFrame and merge with original DataFrame
        df_books = pd.json_normalize(df_exploded['books'])
        # Combining the two DataFrames
        df = pd.concat([df_exploded.drop(columns=['books']), df_books], axis=1)
        # Processing buy_links column
        # For each dictionary "link" in list "x" create a new dictionary where key is the value of "name" and value is the value of "url"
        df_buy_links = pd.json_normalize(df['buy_links'].apply(lambda x: {link['name']: link['url'] for link in x}))
        # Process isbns column
        df_isbns = pd.json_normalize(df['isbns'].apply(lambda x: x[0]))
        # Concatenate processed columns with the original DataFrame
        df = pd.concat([df.drop(['buy_links', 'isbns'], axis=1), df_buy_links, df_isbns], axis=1)
        return df
    else:
        print("Failed to fetch the best list by date data.")
        return None

In [86]:
#Function call
best_list_by_date_df = print_best_list_by_date(list_name="hardcover-fiction", published_date="2024-01-28")
best_list_by_date_df

,bestsellers_date,published_date,published_date_description,next_published_date,previous_published_date,display_name,normal_list_ends_at,updated,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,book_uri,Amazon,Apple Books,Barnes and Noble,Books-A-Million,Bookshop,IndieBound,isbn10,isbn13
0,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,1,1,36,0,0,1649374046,9781649374042,Red Tower,Violet Sorrengail is urged by the commanding g...,0.00,FOURTH WING,Rebecca Yarros,by Rebecca Yarros,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374046?tag=thene...,,,,,,nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a,https://www.amazon.com/dp/1649374046?tag=thene...,https://goto.applebooks.apple/9781649374042?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374042,https://www.indiebound.org/book/9781649374042?...,1649374046,9781649374042
1,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,2,2,10,0,0,1649374178,9781649374172,Red Tower,The second book in the Empyrean series. Violet...,0.00,IRON FLAME,Rebecca Yarros,by Rebecca Yarros,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374178?tag=thene...,,,,,,nyt://book/d3c570c9-3c3a-5c8b-a740-85ea5e92bfc9,https://www.amazon.com/dp/1649374178?tag=thene...,https://goto.applebooks.apple/9781649374172?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374172,https://www.indiebound.org/book/9781649374172?...,1649374178,9781649374172
2,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,3,0,1,0,0,1649374119,9781649374110,Red Tower,When Harrow takes an interest in a winged beas...,0.00,SANCTUARY OF THE SHADOW,Aurora Ascher,by Aurora Ascher,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374119?tag=thene...,,,,,,nyt://book/210bcb4e-8cd2-5583-8bb0-b84846174642,https://www.amazon.com/dp/1649374119?tag=thene...,https://goto.applebooks.apple/9781649374110?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374110,https://www.indiebound.org/book/9781649374110?...,1649374119,9781649374110
3,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,4,3,21,0,0,0593422945,9780593422946,Riverhead,Secrets held by the residents of a dilapidated...,0.00,THE HEAVEN & EARTH GROCERY STORE,James McBride,by James McBride,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/0593422945?tag=thene...,,,,,,nyt://book/7d47b076-7ad7-539c-a051-6664384636b9,https://www.amazon.com/dp/0593422945?tag=thene...,https://goto.applebooks.apple/9780593422946?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780593422946,https://www.indiebound.org/book/9780593422946?...,0593422945,9780593422946
4,2024-01-13,2024-01-28,,2024-02-04,2024-01-21,Hardcover Fiction,15,WEEKLY,5,5,2,0,0,0593492919,9780593492918,Pamela Dorman,A woman who works for a mysterious boss takes ...,0.00,FIRST LIE WINS,Ashley Elston,by Ashley Elston,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/0593492919?tag=thene...,,,,,,nyt://book/d5aa005c-08a5-5bd9-ba0e-6dcd5e7e31f8,https://www.amazon.com/dp/0593492919?tag=thene...,https://goto.applebooks.apple/9780593492918?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780593492918,htt

---

### Best Sellers History
**-------------------------------------**

Get Best Sellers list history.
> ***/lists/best-sellers/history.json***

Parmeters:

**age-group(string)**-The target age group for the best seller.

**author(string)**- The author of the best seller. The author field does not include additional contributors.When searching the author field, you can specify any combination of first, middle and last names.When sort-by is set to author, the results will be sorted by author's first name.

**contributor(string)**-The author of the best seller, as well as other contributors such as the illustrator (to search or sort by author name only, use author instead). When searching, you can specify any combination of first, middle and last names of any of the contributors. When sort-by is set to contributor, the results will be sorted by the first name of the first contributor listed.

**isbn(string)**-International Standard Book Number, 10 or 13 digits.A best seller may have both 10-digit and 13-digit ISBNs, and may have multiple ISBNs of each type. To search on multiple ISBNs, separate the ISBNs with semicolons (example: 9780446579933;0061374229).

**offset(integer)**-must be a multiple of 20.

**price(string)**-The publisher's list price of the best seller, including decimal point.

**publisher(string)**-The standardized name of the publisher

**title(string)**-The title of the best seller. When searching, you can specify a portion of a title or a full title.

In [51]:
#Function to make API call
def get_best_list_history(age_group=None, author=None, contributor=None,isbn=None,offset=None,price=None,publisher=None,title=None):
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json"    
    #API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"    
    # Request parameters
    params = {
        "api-key": api_key
    }
    if age_group:
        params["age-group"] = age_group
    if author:
        params["author"] = author
    if contributor:
        params["contributor"] = contributor
    if isbn:
        params["isbn"] = isbn
    if offset is not None:
        params["offset"] = offset
    if price:
        params["price"] = price
    if publisher:
        params["publisher"] = publisher
    if title:
        params["title"] = title   
    # Making the API request
    response = requests.get(base_url, params=params)    
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [52]:
#Function call
best_list_history = get_best_list_history(author="Sophia Amoruso")
best_list_history

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 1,
 'results': [{'title': '#GIRLBOSS',
   'description': 'An online fashion retailer traces her path to success.',
   'contributor': 'by Sophia Amoruso',
   'author': 'Sophia Amoruso',
   'contributor_note': '',
   'price': '0.00',
   'age_group': '',
   'publisher': 'Portfolio/Penguin/Putnam',
   'isbns': [{'isbn10': '039916927X', 'isbn13': '9780399169274'},
    {'isbn10': '1591847931', 'isbn13': '9781591847939'}],
   'ranks_history': [{'primary_isbn10': '1591847931',
     'primary_isbn13': '9781591847939',
     'rank': 8,
     'list_name': 'Business Books',
     'display_name': 'Business',
     'published_date': '2016-03-13',
     'bestsellers_date': '2016-02-27',
     'weeks_on_list': 0,
     'rank_last_week': 0,
     'asterisk': 0,
     'dagger': 0},
    {'primary_isbn10': '1591847931',
     'primary_isbn13': '9781591847939',
     'rank': 9,
     'list_name': 'Busi

Converting to a dataframe

In [55]:
# Extracting the 'results' data
results_data = best_list_history.get('results', [])
# Converting to DataFrame
df = pd.DataFrame(results_data)
l10=[]
l13=[]
for d in df["isbns"][0]:
    if d["isbn10"]:
        l10.append(d["isbn10"])
    if d["isbn13"]:
        l13.append(d["isbn13"])
# Creating a  dataFrame with lists of ISBN10 and ISBN13 under separate columns
df_isbns = pd.DataFrame({'isbn10': [l10], 'isbn13': [l13]})
df = pd.concat([df, df_isbns], axis=1)
df.drop(columns="isbns",inplace=True)
# Extracting  into separate columns
df = pd.concat([df.drop(['reviews'], axis=1), pd.json_normalize(df['reviews'].apply(lambda x: x[0]))], axis=1)
# Exploding ranks_history list of dictionaries into separate rows
df_exploded = df.explode('ranks_history')
df_exploded.reset_index(inplace=True)
df_exploded.drop(columns="index",inplace=True)
#Converting ranks_history back to DataFrame and merge with original DataFrame
df_ranks_history = pd.json_normalize(df_exploded['ranks_history'])
# Combining the two dataFrames
df_combined = pd.concat([df_exploded.drop(columns=['ranks_history']), df_ranks_history], axis=1)
df_combined.head(2)

,title,description,contributor,author,contributor_note,price,age_group,publisher,isbn10,isbn13,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,primary_isbn10,primary_isbn13,rank,list_name,display_name,published_date,bestsellers_date,weeks_on_list,rank_last_week,asterisk,dagger
0,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,1591847931,9781591847939,8,Business Books,Business,2016-03-13,2016-02-27,0,0,0,0
1,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,1591847931,9781591847939,9,Business Books,Business,2016-01-17,2016-01-02,0,0,0,0


Making a function

In [56]:
def print_best_list_history(age_group=None, author=None, contributor=None, isbn=None, offset=None, price=None, publisher=None, title=None):
    # Get best list history data
    best_list_history_data = get_best_list_history(age_group, author, contributor, isbn, offset, price, publisher, title)
    if best_list_history_data:
        # Extracting the 'results' data
        results_data = best_list_history_data.get('results', [])
        # Converting to DataFrame
        df = pd.DataFrame(results_data)
        l10=[]
        l13=[]
        for d in df["isbns"][0]:
            if d["isbn10"]:
                l10.append(d["isbn10"])
            if d["isbn13"]:
                l13.append(d["isbn13"])
        # Creating a DataFrame with lists of ISBN10 and ISBN13 under separate columns
        df_isbns = pd.DataFrame({'isbn10': [l10], 'isbn13': [l13]})
        df = pd.concat([df, df_isbns], axis=1)
        df.drop(columns="isbns", inplace=True)
        # Extracting  into separate columns
        df = pd.concat([df.drop(['reviews'], axis=1), pd.json_normalize(df['reviews'].apply(lambda x: x[0]))], axis=1)
        # Exploding ranks_history list of dictionaries into separate rows
        df_exploded = df.explode('ranks_history')
        df_exploded.reset_index(inplace=True)
        df_exploded.drop(columns="index", inplace=True)
        # Converting ranks_history back to DataFrame and merge with the original DataFrame
        df_ranks_history = pd.json_normalize(df_exploded['ranks_history'])
        # Combining the two DataFrames
        df_combined = pd.concat([df_exploded.drop(columns=['ranks_history']), df_ranks_history], axis=1)
        return df_combined
    else:
        print("Failed to fetch the best list history data.")
        return None

In [57]:
#Function call
best_list_history_df = print_best_list_history(author="Sophia Amoruso")
best_list_history_df

,title,description,contributor,author,contributor_note,price,age_group,publisher,isbn10,isbn13,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,primary_isbn10,primary_isbn13,rank,list_name,display_name,published_date,bestsellers_date,weeks_on_list,rank_last_week,asterisk,dagger
0,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,1591847931,9781591847939,8,Business Books,Business,2016-03-13,2016-02-27,0,0,0,0
1,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,1591847931,9781591847939,9,Business Books,Business,2016-01-17,2016-01-02,0,0,0,0
2,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,1591847931,9781591847939,9,Business Books,Business,2015-12-13,2015-11-28,0,0,0,0
3,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,1591847931,9781591847939,8,Business Books,Business,2015-11-15,2015-10-31,0,0,0,0
4,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,039916927X,9780399169274,10,Business Books,Business,2014-11-09,2014-10-25,0,0,0,0
5,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,039916927X,9780399169274,8,Business Books,Business,2014-10-12,2014-09-27,0,0,0,0
6,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,039916927X,9780399169274,14,Advice How-To and Miscellaneous,"Advice, How-To & Miscellaneous",2014-09-21,2014-09-06,0,0,0,0
7,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,039916927X,9780399169274,13,Advice How-To and Miscellaneous,"Advice, How-To & Miscellaneous",2014-09-14,2014-08-30,0,0,0,0
8,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,039916927X,9780399169274,5,Business Books,Business,2014-09-14,2014-08-30,0,0,0,0
9,#GIRLBOSS,An online fashion retailer traces her path to ...,by Sophia Amoruso,Sophia Amoruso,,0.00,,Portfolio/Penguin/Putnam,"[039916927X, 1591847931]","[9780399169274, 9781591847939]",,,,,039916927X,9780399169274,8,Advice How-To and Miscellaneous,"Advice, How-To & Miscellaneous",2014-09-07,2014-08-23,16,0,0,0


### Best Seller Full Overview
**----------------------------------------------**

Get all books for all the Best Sellers lists for specified date.

>***/lists/full-overview.json***


Parmeters:

**published_date(string, YYYY-MM-DD)**- The best-seller list publication date. You do not have to specify the exact date the list was published. The service will search forward (into the future) for the closest publication date to the date you specify. For example, a request for lists/overview/2013-05-22 will retrieve the list that was published on 05-26. If you do not include a published date, the current week's best sellers lists will be returned.

In [44]:
#Function to make API call
def get_full_overview(published_date=None):        
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/lists/full-overview.json"    
    #API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"    
    # Request parameters
    params = {
        "api-key": api_key
    }    
    if published_date:
        params["published_date"] = published_date    
    # Making the API request
    response = requests.get(base_url, params=params)    
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [45]:
#Function call
best_list_full_overview = get_full_overview(published_date="2024-01-01")
best_list_full_overview

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 230,
 'results': {'bestsellers_date': '2023-12-23',
  'published_date': '2024-01-07',
  'published_date_description': '',
  'previous_published_date': '2023-12-31',
  'next_published_date': '2024-01-14',
  'lists': [{'list_id': 704,
    'list_name': 'Combined Print and E-Book Fiction',
    'list_name_encoded': 'combined-print-and-e-book-fiction',
    'display_name': 'Combined Print & E-Book Fiction',
    'updated': 'WEEKLY',
    'list_image': None,
    'list_image_width': None,
    'list_image_height': None,
    'books': [{'age_group': '',
      'amazon_product_url': 'https://www.amazon.com/dp/1649374046?tag=thenewyorktim-20',
      'article_chapter_link': '',
      'author': 'Rebecca Yarros',
      'book_image': 'https://storage.googleapis.com/du-prd/books/images/9781649374042.jpg',
      'book_image_width': 309,
      'book_image_height': 500,
      'book_review_link

Converting to dataframe

In [48]:
# Extracting the 'results' data
results_data = best_list_full_overview.get('results', [])
# Converting to DataFrame
df = pd.DataFrame(results_data)
# Extracting  into separate columns
df = pd.concat([df.drop(['lists'], axis=1), pd.json_normalize(df['lists'].apply(lambda x: x))], axis=1)
# Exploding ranks_history list of dictionaries into separate rows
df = df.explode('books')
df.reset_index(inplace=True)
df.drop(columns="index",inplace=True)
# Extracting  into separate columns
df = pd.concat([df.drop(['books'], axis=1), pd.json_normalize(df['books'].apply(lambda x: x))], axis=1)
# Processing buy_links column
#For each dictionary "link" in list "x"  create a new dictionary where key is the value of "name" and value is the value of "url"
df = pd.concat([df.drop(['buy_links'], axis=1), pd.json_normalize(df['buy_links'].apply(lambda x: {link['name']: link['url'] for link in x}))], axis=1)
df.head(2)

,bestsellers_date,published_date,published_date_description,previous_published_date,next_published_date,list_id,list_name,list_name_encoded,display_name,updated,list_image,list_image_width,list_image_height,age_group,amazon_product_url,article_chapter_link,author,book_image,book_image_width,book_image_height,book_review_link,book_uri,btrn,contributor,contributor_note,created_date,description,first_chapter_link,price,primary_isbn10,primary_isbn13,publisher,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,Amazon,Apple Books,Barnes and Noble,Books-A-Million,Bookshop,IndieBound
0,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1649374046?tag=thene...,,Rebecca Yarros,https://storage.googleapis.com/du-prd/books/im...,309,500,,nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a,222848047,by Rebecca Yarros,,2023-12-27 23:20:33,Violet Sorrengail is urged by the commanding g...,,0.00,1649374046,9781649374042,Red Tower,1,1,,FOURTH WING,2023-12-27 23:25:38,34,https://www.amazon.com/dp/1649374046?tag=thene...,https://goto.applebooks.apple/9781649374042?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374042,https://www.indiebound.org/book/9781649374042?...
1,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1649374178?tag=thene...,,Rebecca Yarros,https://storage.googleapis.com/du-prd/books/im...,309,500,,nyt://book/d3c570c9-3c3a-5c8b-a740-85ea5e92bfc9,225103271,by Rebecca Yarros,,2023-12-27 23:20:33,The second book in the Empyrean series. Violet...,,0.00,1649374178,9781649374172,Red Tower,2,2,,IRON FLAME,2023-12-27 23:25:38,7,https://www.amazon.com/dp/1649374178?tag=thene...,https://goto.applebooks.apple/9781649374172?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374172,https://www.indiebound.org/book/9781649374172?...


Putting it all into a function

In [87]:
def print_full_overview(published_date=None):
    # Get full overview data
    full_overview_data = get_full_overview(published_date)
    if full_overview_data:
        # Extracting the 'results' data
        results_data = full_overview_data.get('results', [])
        # Converting to DataFrame
        df = pd.DataFrame(results_data)
        # Extracting  into separate columns
        df = pd.concat([df.drop(['lists'], axis=1), pd.json_normalize(df['lists'].apply(lambda x: x))], axis=1)
        # Exploding ranks_history list of dictionaries into separate rows
        df = df.explode('books')
        df.reset_index(inplace=True)
        df.drop(columns="index", inplace=True)
        # Extracting  into separate columns
        df = pd.concat([df.drop(['books'], axis=1), pd.json_normalize(df['books'].apply(lambda x: x))], axis=1)
        # Processing buy_links column
        # For each dictionary "link" in list "x" create a new dictionary where key is the value of "name" and value is the value of "url"
        df = pd.concat([df.drop(['buy_links'], axis=1), pd.json_normalize(df['buy_links'].apply(lambda x: {link['name']: link['url'] for link in x}))], axis=1)
        return df
    else:
        print("Failed to fetch the full overview data.")
        return None

In [88]:
#Function call
full_overview_df = print_full_overview(published_date="2024-01-01")
full_overview_df

,bestsellers_date,published_date,published_date_description,previous_published_date,next_published_date,list_id,list_name,list_name_encoded,display_name,updated,list_image,list_image_width,list_image_height,age_group,amazon_product_url,article_chapter_link,author,book_image,book_image_width,book_image_height,book_review_link,book_uri,btrn,contributor,contributor_note,created_date,description,first_chapter_link,price,primary_isbn10,primary_isbn13,publisher,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,Amazon,Apple Books,Barnes and Noble,Books-A-Million,Bookshop,IndieBound
0,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1649374046?tag=thene...,,Rebecca Yarros,https://storage.googleapis.com/du-prd/books/im...,309,500,,nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a,222848047,by Rebecca Yarros,,2023-12-27 23:20:33,Violet Sorrengail is urged by the commanding g...,,0.00,1649374046,9781649374042,Red Tower,1,1,,FOURTH WING,2023-12-27 23:25:38,34,https://www.amazon.com/dp/1649374046?tag=thene...,https://goto.applebooks.apple/9781649374042?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374042,https://www.indiebound.org/book/9781649374042?...
1,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1649374178?tag=thene...,,Rebecca Yarros,https://storage.googleapis.com/du-prd/books/im...,309,500,,nyt://book/d3c570c9-3c3a-5c8b-a740-85ea5e92bfc9,225103271,by Rebecca Yarros,,2023-12-27 23:20:33,The second book in the Empyrean series. Violet...,,0.00,1649374178,9781649374172,Red Tower,2,2,,IRON FLAME,2023-12-27 23:25:38,7,https://www.amazon.com/dp/1649374178?tag=thene...,https://goto.applebooks.apple/9781649374172?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374172,https://www.indiebound.org/book/9781649374172?...
2,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/0593422945?tag=thene...,,James McBride,https://storage.googleapis.com/du-prd/books/im...,331,500,,nyt://book/7d47b076-7ad7-539c-a051-6664384636b9,224332678,by James McBride,,2023-12-27 23:20:33,Secrets held by the residents of a dilapidated...,,0.00,0593422945,9780593422946,Riverhead,3,3,,THE HEAVEN & EARTH GROCERY STORE,2023-12-27 23:25:38,8,https://www.amazon.com/dp/0593422945?tag=thene...,https://goto.applebooks.apple/9780593422946?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780593422946,https://www.indiebound.org/book/9780593422946?...
3,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/0385548958?tag=thene...,,John Grisham,https://storage.googleapis.com/du-prd/books/im...,329,500,,nyt://book/52c31163-754f-58c8-90d8-ea1f297f7030,225918462,by John Grisham,,2023-12-27 23:20:33,"In a sequel to “The Firm,” Mitch McDeere, who ...",,0.00,0385548958,9780385548953,Doubleday,4,4,,THE EXCHANGE,2023-12-27 23:25:38,10,https://www.amazon.com/dp/0385548958?tag=thene...,https://goto.applebooks.apple/9780385548953?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780385548953,https://www.indiebound.org/book/9780385548953?...
4,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e

---

### Best Sellers List Names
**-------------------------------------------**

Get Best Sellers list names.

> ***/lists/names.json***


No method level parameters


In [38]:
#Function to make the API call
def get_names(published_date=None):        
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/lists/names.json"    
    #API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"    
    # Request parameters
    params = {
        "api-key": api_key
    } 
    # Making the API request
    response = requests.get(base_url, params=params)    
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [39]:
#Function call
name_list_history = get_names(published_date="2024-01-01")
name_list_history

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 59,
 'results': [{'list_name': 'Combined Print and E-Book Fiction',
   'display_name': 'Combined Print & E-Book Fiction',
   'list_name_encoded': 'combined-print-and-e-book-fiction',
   'oldest_published_date': '2011-02-13',
   'newest_published_date': '2024-05-12',
   'updated': 'WEEKLY'},
  {'list_name': 'Combined Print and E-Book Nonfiction',
   'display_name': 'Combined Print & E-Book Nonfiction',
   'list_name_encoded': 'combined-print-and-e-book-nonfiction',
   'oldest_published_date': '2011-02-13',
   'newest_published_date': '2024-05-12',
   'updated': 'WEEKLY'},
  {'list_name': 'Hardcover Fiction',
   'display_name': 'Hardcover Fiction',
   'list_name_encoded': 'hardcover-fiction',
   'oldest_published_date': '2008-06-08',
   'newest_published_date': '2024-05-12',
   'updated': 'WEEKLY'},
  {'list_name': 'Hardcover Nonfiction',
   'display_name': 'Hardcover No

Converting to a dataframe

In [194]:
# Extracting the 'results' data
results_data = name_list_history.get('results', [])
# Converting to DataFrame
df = pd.DataFrame(results_data)
df.head(2)

,list_name,display_name,list_name_encoded,oldest_published_date,newest_published_date,updated
0,Combined Print and E-Book Fiction,Combined Print & E-Book Fiction,combined-print-and-e-book-fiction,2011-02-13,2024-05-05,WEEKLY
1,Combined Print and E-Book Nonfiction,Combined Print & E-Book Nonfiction,combined-print-and-e-book-nonfiction,2011-02-13,2024-05-05,WEEKLY
2,Hardcover Fiction,Hardcover Fiction,hardcover-fiction,2008-06-08,2024-05-05,WEEKLY
3,Hardcover Nonfiction,Hardcover Nonfiction,hardcover-nonfiction,2008-06-08,2024-05-05,WEEKLY
4,Trade Fiction Paperback,Paperback Trade Fiction,trade-fiction-paperback,2008-06-08,2024-05-05,WEEKLY
5,Mass Market Paperback,Paperback Mass-Market Fiction,mass-market-paperback,2008-06-08,2017-01-29,WEEKLY
6,Paperback Nonfiction,Paperback Nonfiction,paperback-nonfiction,2008-06-08,2024-05-05,WEEKLY
7,E-Book Fiction,E-Book Fiction,e-book-fiction,2011-02-13,2017-01-29,WEEKLY
8,E-Book Nonfiction,E-Book Nonfiction,e-book-nonfiction,2011-02-13,2017-01-29,WEEKLY
9,Hardcover Advice,Hardcover Advice & Misc.,hardcover-advice,2008-06-08,2013-04-21,WEEKLY


Making it a single function

In [42]:
#Function to print the list
def print_names(published_date=None):
    # Get names data
    names_data = get_names(published_date)
    if names_data:
        # Extracting the 'results' data
        results_data = names_data.get('results', [])
        # Converting to DataFrame
        df = pd.DataFrame(results_data)
        return df
    else:
        print("Failed to fetch the names data.")
        return None

In [43]:
#Function call
names_df = print_names()
names_df

,list_name,display_name,list_name_encoded,oldest_published_date,newest_published_date,updated
0,Combined Print and E-Book Fiction,Combined Print & E-Book Fiction,combined-print-and-e-book-fiction,2011-02-13,2024-05-12,WEEKLY
1,Combined Print and E-Book Nonfiction,Combined Print & E-Book Nonfiction,combined-print-and-e-book-nonfiction,2011-02-13,2024-05-12,WEEKLY
2,Hardcover Fiction,Hardcover Fiction,hardcover-fiction,2008-06-08,2024-05-12,WEEKLY
3,Hardcover Nonfiction,Hardcover Nonfiction,hardcover-nonfiction,2008-06-08,2024-05-12,WEEKLY
4,Trade Fiction Paperback,Paperback Trade Fiction,trade-fiction-paperback,2008-06-08,2024-05-12,WEEKLY
5,Mass Market Paperback,Paperback Mass-Market Fiction,mass-market-paperback,2008-06-08,2017-01-29,WEEKLY
6,Paperback Nonfiction,Paperback Nonfiction,paperback-nonfiction,2008-06-08,2024-05-12,WEEKLY
7,E-Book Fiction,E-Book Fiction,e-book-fiction,2011-02-13,2017-01-29,WEEKLY
8,E-Book Nonfiction,E-Book Nonfiction,e-book-nonfiction,2011-02-13,2017-01-29,WEEKLY
9,Hardcover Advice,Hardcover Advice & Misc.,hardcover-advice,2008-06-08,2013-04-21,WEEKLY


## Top 5 Best Sellers list for Specified date (Overview)

Get top 5 books for all the Best Sellers lists for specified date.

> ***/lists/overview.json***

Paramters:

**published_date(string,YYYY-MM-DD)** - The best-seller list publication date. You do not have to specify the exact date the list was published. The service will search forward (into the future) for the closest publication date to the date you specify. For example, a request for lists/overview/2013-05-22 will retrieve the list that was published on 05-26.If you do not include a published date, the current week's best sellers lists will be returned.

In [31]:
#Function to make API call
def get_overview(published_date=None):  
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/lists/overview.json"
    #API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"
    # Request parameters
    params = {
        "api-key": api_key
    }
    if published_date:
        params["published_date"] = published_date
    # Making the API request
    response = requests.get(base_url, params=params)
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [32]:
best_list_overview = get_overview(published_date="2024-01-01")
best_list_overview

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 90,
 'results': {'bestsellers_date': '2023-12-23',
  'published_date': '2024-01-07',
  'published_date_description': '',
  'previous_published_date': '2023-12-31',
  'next_published_date': '2024-01-14',
  'lists': [{'list_id': 704,
    'list_name': 'Combined Print and E-Book Fiction',
    'list_name_encoded': 'combined-print-and-e-book-fiction',
    'display_name': 'Combined Print & E-Book Fiction',
    'updated': 'WEEKLY',
    'list_image': None,
    'list_image_width': None,
    'list_image_height': None,
    'books': [{'age_group': '',
      'amazon_product_url': 'https://www.amazon.com/dp/1649374046?tag=thenewyorktim-20',
      'article_chapter_link': '',
      'author': 'Rebecca Yarros',
      'book_image': 'https://storage.googleapis.com/du-prd/books/images/9781649374042.jpg',
      'book_image_width': 309,
      'book_image_height': 500,
      'book_review_link'

In [33]:
# Extracting the 'results' data
results_data = best_list_overview.get('results', [])
# Converting to DataFrame
df = pd.DataFrame(results_data)
# Extracting  into separate columns
df = pd.concat([df.drop(['lists'], axis=1), pd.json_normalize(df['lists'].apply(lambda x: x))], axis=1)
# Exploding 'books' list of dictionaries into separate rows
df = df.explode('books')
df.reset_index(inplace=True)
df.drop(columns="index",inplace=True)
# Extracting 'books'  into separate columns
df = pd.concat([df.drop(['books'], axis=1), pd.json_normalize(df['books'].apply(lambda x: x))], axis=1)
# Processing the buy_links column
#For each dictionary "link" in list "x"  creating a new dictionary where key is the value of "name" and value is the value of "url"
df = pd.concat([df.drop(['buy_links'], axis=1), pd.json_normalize(df['buy_links'].apply(lambda x: {link['name']: link['url'] for link in x}))], axis=1)
df.head(2)

,bestsellers_date,published_date,published_date_description,previous_published_date,next_published_date,list_id,list_name,list_name_encoded,display_name,updated,list_image,list_image_width,list_image_height,age_group,amazon_product_url,article_chapter_link,author,book_image,book_image_width,book_image_height,book_review_link,contributor,contributor_note,created_date,description,first_chapter_link,price,primary_isbn10,primary_isbn13,book_uri,publisher,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,Amazon,Apple Books,Barnes and Noble,Books-A-Million,Bookshop,IndieBound
0,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1649374046?tag=thene...,,Rebecca Yarros,https://storage.googleapis.com/du-prd/books/im...,309,500,,by Rebecca Yarros,,2023-12-27 23:20:33,Violet Sorrengail is urged by the commanding g...,,0.00,1649374046,9781649374042,nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a,Red Tower,1,1,,FOURTH WING,2023-12-27 23:25:38,34,https://www.amazon.com/dp/1649374046?tag=thene...,https://goto.applebooks.apple/9781649374042?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374042,https://www.indiebound.org/book/9781649374042?...
1,2023-12-23,2024-01-07,,2023-12-31,2024-01-14,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1649374178?tag=thene...,,Rebecca Yarros,https://storage.googleapis.com/du-prd/books/im...,309,500,,by Rebecca Yarros,,2023-12-27 23:20:33,The second book in the Empyrean series. Violet...,,0.00,1649374178,9781649374172,nyt://book/d3c570c9-3c3a-5c8b-a740-85ea5e92bfc9,Red Tower,2,2,,IRON FLAME,2023-12-27 23:25:38,7,https://www.amazon.com/dp/1649374178?tag=thene...,https://goto.applebooks.apple/9781649374172?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781649374172,https://www.indiebound.org/book/9781649374172?...


Putting it together

In [36]:
def print_overview(published_date=None):
    # Overview data
    overview_data = get_overview(published_date)
    if overview_data:
        # Extracting the 'results' data
        results_data = overview_data.get('results', [])
        # Converting to DataFrame
        df = pd.DataFrame(results_data)
        # Extracting  into separate columns
        df = pd.concat([df.drop(['lists'], axis=1), pd.json_normalize(df['lists'].apply(lambda x: x))], axis=1)
        # Exploding 'books' list of dictionaries into separate rows
        df = df.explode('books')
        df.reset_index(inplace=True)
        df.drop(columns="index", inplace=True)
        # Extracting 'books'  into separate columns
        df = pd.concat([df.drop(['books'], axis=1), pd.json_normalize(df['books'].apply(lambda x: x))], axis=1)
        # Processing the buy_links column
        # For each dictionary "link" in list "x"  creating a new dictionary where key is the value of "name" and value is the value of "url"
        df = pd.concat([df.drop(['buy_links'], axis=1), pd.json_normalize(df['buy_links'].apply(lambda x: {link['name']: link['url'] for link in x}))], axis=1)
        # Display the DataFrame
        return df
    else:
        print("Failed to fetch the overview data.")
        return None

In [37]:
print_overview(published_date="2024-04-28")

,bestsellers_date,published_date,published_date_description,previous_published_date,next_published_date,list_id,list_name,list_name_encoded,display_name,updated,list_image,list_image_width,list_image_height,age_group,amazon_product_url,article_chapter_link,author,book_image,book_image_width,book_image_height,book_review_link,contributor,contributor_note,created_date,description,first_chapter_link,price,primary_isbn10,primary_isbn13,book_uri,publisher,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,Amazon,Apple Books,Barnes and Noble,Books-A-Million,Bookshop,IndieBound
0,2024-04-13,2024-04-28,,2024-04-21,2024-05-05,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/0593714490?tag=thene...,,John Sandford,https://storage.googleapis.com/du-prd/books/im...,331,500,,by John Sandford,,2024-04-17 22:20:34,The 34th book in the Prey series. Letty and Lu...,,0.00,0593714504,9780593714508,nyt://book/6839ca12-8968-500f-a7ee-3832cc77d3c0,Putnam,1,0,,TOXIC PREY,2024-04-17 22:25:37,1,https://www.amazon.com/dp/0593714490?tag=thene...,https://goto.applebooks.apple/9780593714508?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780593714508,https://www.indiebound.org/book/9780593714508?...
1,2024-04-13,2024-04-28,,2024-04-21,2024-05-05,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/1250178630?tag=thene...,,Kristin Hannah,https://storage.googleapis.com/du-prd/books/im...,333,500,,by Kristin Hannah,,2024-04-17 22:20:34,"In 1965, a nursing student follows her brother...",,0.00,1250178630,9781250178633,nyt://book/9ce735af-71cf-5ff3-a367-43ee07e3fdd7,St. Martin's,2,2,,THE WOMEN,2024-04-17 22:25:37,10,https://www.amazon.com/dp/1250178630?tag=thene...,https://goto.applebooks.apple/9781250178633?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781250178633,https://www.indiebound.org/book/9781250178633?...
2,2024-04-13,2024-04-28,,2024-04-21,2024-05-05,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,https://www.amazon.com/dp/125088425X?tag=thene...,,Leigh Bardugo,https://storage.googleapis.com/du-prd/books/im...,350,500,,by Leigh Bardugo,,2024-04-17 22:20:34,Luzia Cotado encounters dangers when her magic...,,0.00,125088425X,9781250884251,nyt://book/44383776-4732-5e8e-8b32-50c324d1b120,Flatiron,3,0,,THE FAMILIAR,2024-04-17 22:25:37,1,https://www.amazon.com/dp/125088425X?tag=thene...,https://goto.applebooks.apple/9781250884251?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781250884251,https://www.indiebound.org/book/9781250884251?...
3,2024-04-13,2024-04-28,,2024-04-21,2024-05-05,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,None,None,None,,http://www.amazon.com/Court-Thorns-Roses-Sarah...,,Sarah J. Maas,https://storage.googleapis.com/du-prd/books/im...,326,495,,by Sarah J. Maas,,2024-04-17 22:20:34,"After killing a wolf in the woods, Feyre is ta...",,0.00,1635575567,9781635575569,nyt://book/cc163d0f-b8a2-573c-8e81-d804ac7844af,Bloomsbury,4,0,,A COURT OF THORNS AND ROSES,2024-04-17 22:25:37,1,http://www.amazon.com/Court-Thorns-Roses-Sarah...,https://goto.applebooks.apple/9781635575569?at...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9781635575569,https://www.indiebound.org/book/9781635575569?...
4,2024-04-13,2024-04-28,,2024-04-21,2024-05-05,704,Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKL

## Reviews

The API doesn't provide some of the reviews.

Get book reviews
>***/reviews.json***

Parmeters:
    
**isbn(integer)**: Searching by ISBN is the recommended method. You can enter 10- or 13-digit ISBNs.

**title(string)**: You’ll need to enter the full title of the book. Spaces in the title will be converted into the characters %20.

**author(string)**: You’ll need to enter the author’s first and last name, separated by a space. This space will be converted into the characters %20.

*Note:*

*In URLs, spaces are not allowed, and instead, they are encoded as %20. This is part of URL encoding, which replaces reserved characters with a percent sign (%) followed by their ASCII hexadecimal value.For instance, if you have an author's name like "John Smith," the space between "John" and "Smith" needs to be replaced with %20. So, the URL-encoded version of "John Smith" would be "John%20Smith" in the URL.*

In [101]:
#Function to make API call
def get_review(isbn=None,title=None,author=None):  
    # Base URL for the API
    base_url = "https://api.nytimes.com/svc/books/v3/reviews.json"
    #API key
    api_key="VSnHzkRDYazY8vpsaGWhbSNy0MM5Rfoj"
    # Request parameters
    params = {
        "api-key": api_key
    }
    if isbn:
        params["isbn"] = isbn
    if title:
        params["title"] = title
    if author:
        params["author"] = author        
    # Making the API request
    response = requests.get(base_url, params=params)
    # Checking if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [141]:
#Function call
review = get_review(author="Michelle Obama")
review

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.',
 'num_results': 1,
 'results': [{'url': 'https://www.nytimes.com/2018/12/06/books/review/michelle-obama-becoming-memoir.html',
   'publication_dt': '2018-12-06',
   'byline': 'Isabel Wilkerson',
   'book_title': 'Becoming',
   'book_author': 'Michelle Obama',
   'summary': 'The former first lady’s long-awaited new memoir recounts with insight, candor and wit her family’s trajectory from the Jim Crow South to Chicago’s South Side and her own improbable journey from there to the White House.',
   'uuid': '00000000-0000-0000-0000-000000000000',
   'uri': 'nyt://book/00000000-0000-0000-0000-000000000000',
   'isbn13': ['9781524763138']}]}

Converting to Dataframe

In [128]:
# Extracting the 'results' data
results_data = review.get('results', [])
# Converting to DataFrame
df = pd.DataFrame(results_data)
df.head()

,url,publication_dt,byline,book_title,book_author,summary,uuid,uri,isbn13
0,https://www.nytimes.com/2018/12/06/books/revie...,2018-12-06,Isabel Wilkerson,Becoming,Michelle Obama,The former first lady’s long-awaited new memoi...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9781524763138]


In [131]:
df["url"]

0    https://www.nytimes.com/2018/12/06/books/revie...
Name: url, dtype: object

In [125]:
#Function to print
def print_review(isbn=None, title=None, author=None):
    # Get review data
    review_data = get_review(isbn, title, author)
    if review_data:
        # Extracting the 'results' data
        results_data = review_data.get('results', [])
        # Converting to DataFrame
        df = pd.DataFrame(results_data)
        return df
    else:
        print("Failed to fetch the review data.")
        return None

In [137]:
#Function call
review_df = print_review(author="J. K. Rowling")
review_df

,url,publication_dt,byline,book_title,book_author,summary,uuid,uri,isbn13
0,http://www.nytimes.com/2012/10/28/books/review...,2012-10-28,AMANDA FOREMAN,Casual Vacancy,J. K. Rowling,J. K. Rowling turns from children’s fantasy to...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,"[9780316228534, 9780316228541, 9780316228558, ..."
1,http://query.nytimes.com/gst/fullpage.html?res...,2007-08-12,,Harry Potter and the Deathly Hallows,J. K. Rowling,,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9789654826358]
2,http://www.nytimes.com/2007/08/12/books/review...,2007-08-12,CHRISTOPHER HITCHENS,Harry Potter and the Deathly Hallows,J. K. Rowling,J. K. Rowling has won imperishable renown for ...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9780545139700]
3,http://www.nytimes.com/2007/07/19/books/19pott...,2007-07-19,MICHIKO KAKUTANI,Harry Potter and the Deathly Hallows,J. K. Rowling,"J. K. Rowling’s monumental, spell-binding epic...",00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9780545139700]
4,http://www.nytimes.com/2012/09/27/books/book-r...,2012-09-27,MICHIKO KAKUTANI,The Casual Vacancy,J. K. Rowling,The bitter citizens of a small English town ar...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,"[9780316228534, 9780316228541]"
5,https://www.nytimes.com/2012/09/27/books/book-...,2012-09-27,MICHIKO KAKUTANI,The Casual Vacancy,J. K. Rowling,The bitter citizens of a small English town ar...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9780316228589]


In [139]:
review_df["url"][1]

'http://query.nytimes.com/gst/fullpage.html?res=9A02E6DF1630F931A2575BC0A9619C8B63'

In conclusion, we have successfully transformed all the book API URLs into functions that can directly print the DataFrame. Each function takes appropriate parameters, makes the API call, processes the data, and outputs the result as a DataFrame. These functions provide a convenient way to interact with the New York Times Books API and retrieve information about best-sellers, reviews, lists, and more. With these functions in place, users can easily access and analyze book data from the New York Times API.